# MNL

In [11]:
import gurobipy as gp

In [12]:
import numpy as np
import numpy.random as npr
N = 5 #25 #25 50 75 100
M = 1 #1 5 10 20
v = 100*npr.rand(N, M)
theta = 1/M * np.ones(M)
print(v)

[[16.00590565]
 [37.35371385]
 [97.13054119]
 [35.3836815 ]
 [64.36849053]]


In [13]:
def IP_mmnl(p):
    global v, N
    global theta, M

    #create a new model
    myModel = gp.Model("IP_mmnl")

    # create decision variables and store them in the arrays z, x, y
    z = [0 for j in range(M)]
    x = [[0 for j in range(M)] for i in range(N)]
    y = [0 for i in range(N)]
    for j in range(M):
        newVar = myModel.addVar(vtype = gp.GRB.CONTINUOUS, lb = 0, name = "z_" + str(j))
        z[j] = newVar
        for i in range(N):
            newVarx = myModel.addVar(vtype = gp.GRB.CONTINUOUS, lb = 0, name = "x_" + str(i) + '_' + str(j))
            x[i][j] = newVarx
    for i in range(N):
        newVar = myModel.addVar(vtype = gp.GRB.BINARY, name = "y_" + str(i))
        y[i] = newVar
    myModel.update()

    # set objective function
    objExpr = gp.LinExpr()
    for j in range(M):
        objExpr += theta[j] * z[j]
    myModel.setObjective(objExpr, gp.GRB.MAXIMIZE)

    B = 100*max(p)
    # add constraints
    for j in range(M):
        myConstr = gp.LinExpr()
        for i in range(N):
            myConstr += v[i][j] * x[i][j]
            myModel.addConstr(lhs = x[i][j], sense = gp.GRB.LESS_EQUAL, rhs = y[i]*B, name = "x 1st constraint_" + str(i) + '_' + str(j))
            myModel.addConstr(lhs = x[i][j], sense = gp.GRB.GREATER_EQUAL, rhs = v[i][j] - z[j] - (1-y[i])*B, name = "x 2nd constraint_" + str(i) + '_' + str(j))
            myModel.addConstr(lhs = x[i][j], sense = gp.GRB.LESS_EQUAL, rhs = v[i][j] - z[j] + (1-y[i])*B, name = "x 3rd constraint_" + str(i) + '_' + str(j))
        myModel.addConstr(lhs = z[j], sense = gp.GRB.LESS_EQUAL, rhs = myConstr, name = "z " + str(j))

    # solve model
    myModel.optimize()

    bestS = []
    profit = float(myModel.objVal)
    for i in range(N):
        if y[i].x == 1:
            bestS.append(i)
    bestS = sorted(bestS)
    return profit, bestS

In [14]:
def profit_mmnl(p, S):
    global v
    global theta, M
    pi = 0
    for j in range(M):
        Vj_S = 0
        num = 0
        for i in S:
            num += p[i]*v[i][j]
            Vj_S += v[i][j]
        pi += theta[j]*num/(1+Vj_S)
    return pi

In [15]:
def nested_by_price_mmnl(p):
    global N
    profits = [0 for i in range(N+1)]
    S = []
    for i in range(N):
        S.append(i)
        profits[i+1] = profit_mmnl(p, S)
    best_S = [i for i in range(np.argmax(profits))]
    return max(profits), best_S

In [16]:
def find_best_option_mmnl(p, S):
    global v, N
    current_best = profit_mmnl(p, S)
    bestS = S
    flag = 0
    N_S = [k for k in range(N) if k not in S]
    for k in N_S:
        newS = S + [k]
        if profit_mmnl(p, newS) >= current_best:
            flag = 1
            bestS = newS
            current_best = profit_mmnl(p, newS)
    return current_best, bestS, flag

def greedy_mmnl(p):
    global v, N
    max_profit = -1
    best_S = []
    S = []
    flag = 1
    while flag==1:
        max_profit, S, flag = find_best_option_mmnl(p, S)
    best_S = sorted(S)
    return max_profit, best_S

In [17]:
import matplotlib.pyplot as plt
import math
import statistics as st
import time

In [18]:
n_simulations = 10000
avg_price = 100
std_price = 50
#compute prices
p = [0 for i in range(n_simulations)]
for i in range(n_simulations):
    p[i] = abs(npr.normal(avg_price, std_price, N))
    # p[i] = npr.uniform(0, 2*avg_price, N)
    # p[i] = npr.exponential(avg_price, N)

TypeError: abs() takes exactly one argument (2 given)

In [ ]:
# problem solved using the MIP formulation
for i in range(n_simulations):
    start_time = time.time()
    profit, bestS = IP_mmnl(p[i])
    end_time = time.time()
    if i == 0:
        profits_IP = [profit]
        times_IP = [end_time-start_time]
    else:
        profits_IP.append(profit)
        times_IP.append(end_time-start_time)
avg_time_IP = st.mean(times_IP)

Using license file C:\Users\ftibe\gurobi.lic
Academic license - for non-commercial use only


TypeError: 'float' object is not iterable

In [ ]:
# problem solved using the nested formulation
for i in range(n_simulations):
    start_time = time.time()
    profit, bestS = nested_by_price_mmnl(p[i])
    end_time = time.time()
    if i == 0:
        profits_nested = [profit]
        times_nested = [end_time-start_time]
    else:
        profits_nested.append(profit)
        times_nested.append(end_time-start_time)
avg_time_nested = st.mean(times_nested)

In [ ]:
# problem solved using the greedy formulation
for i in range(n_simulations):
    start_time = time.time()
    profit, bestS = greedy_mmnl(p[i])
    end_time = time.time()
    if i == 0:
        profits_greedy = [profit]
        times_greedy = [end_time-start_time]
    else:
        profits_greedy.append(profit)
        times_greedy.append(end_time-start_time)
avg_time_greedy = st.mean(times_greedy)

In [ ]:
# counting how many instances are different
count_nested = 0
different_results_nested = []
count_greedy = 0
different_results_greedy = []
for i in range(n_simulations):
    if profits_IP[i] != profits_greedy[i]:
        count_greedy += 100/n_simulations
        different_results_greedy.append(100*profits_greedy[i]/profits_IP[i])
    if profits_IP[i] != profits_nested[i]:
        count_nested += 100/n_simulations
        different_results_nested.append(100*profits_nested[i]/profits_IP[i])
print('The greedy algorithm is different from the MIP in ' + str(count_greedy) + ' of the instances, \
        with an average ratio of ' + str(st.mean(different_results_greedy)) + '%', \
              'and a minimum ratio of ' + str(min(different_results_greedy)) + '%', \
                'and an average time of ' + str(avg_time_greedy) + ' seconds')
print('The nested algorithm is different from the MIP in ' + str(count_nested) + ' of the instances, \
        with an average ratio of ' + str(st.mean(different_results_nested)) + '%', \
              'and a minimum ratio of ' + str(min(different_results_nested)) + '%', 
                'and an average time of ' + str(avg_time_nested) + ' seconds')
